In [14]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [15]:
empresas = ['AMER3', 'MGLU3', 'PETR4']

In [16]:
def getCsv(empresa):
    dados = pd.read_csv(empresa + '.csv', index_col='Unnamed: 0')
    dados['data_hora'] = pd.to_datetime(dados['data_hora'], format='%Y-%m-%d %H:%M:%S')

    return dados.sort_values('data_hora')

###################################################################################

In [17]:
def getDadosBolsa(empresa, dataMax, dataMin):

    dados = yf.Ticker(empresa + '.SA')
    dados = dados.history(start=(dataMin - timedelta(days=2)), end=(dataMax + timedelta(days=2)), interval='1d')
    dados.reset_index(inplace=True)
    dados['data_hora'] = dados['Date']
    dados.drop(columns=['Date'], inplace=True)
    dados.set_index('data_hora', inplace=True)
    return dados

In [18]:
def getDataMaisProxima(empresa, data):

    datas = {
                abs(data.timestamp() - date.timestamp()) : date for date in empresa.index}
    
    dataMaisProxima = datas[min(datas.keys())]

    return dataMaisProxima - timedelta(days=1), dataMaisProxima, dataMaisProxima + timedelta(days=1), dataMaisProxima + timedelta(days=2)

In [19]:
def getValoresData(dados, data):
    try:
        valores = dados.iloc[dados.index.get_loc(data)]

    except KeyError:
        return {'Abertura' : None,
                'Fechamento' : None}

    return {'Abertura' : valores.Open,
            'Fechamento' : valores.Close}

In [20]:
def extraiValoresBolsa(empresa):
    df = getCsv(empresa)
    
    dados_bolsa = getDadosBolsa(empresa, df['data_hora'].max(), df['data_hora'].min())

    aberturaDiaAnterior = []
    aberturaDiaAtual = []
    aberturaDiaSeguinte = []
    abertura2Dias = []

    fechamentoDiaAnterior = []
    fechamentoDiaAtual = []
    fechamentoDiaSeguinte = []
    fechamento2Dias = []


    for data in df['data_hora']:
        diaAnterior, dataAtual, dataSeguinte, dia2 = getDataMaisProxima(dados_bolsa, data)

        valores = getValoresData(dados_bolsa, diaAnterior)
        aberturaDiaAnterior.append(valores['Abertura'])
        fechamentoDiaAnterior.append(valores['Fechamento'])

        valores = getValoresData(dados_bolsa, dataAtual)
        aberturaDiaAtual.append(valores['Abertura'])
        fechamentoDiaAtual.append(valores['Fechamento'])

        valores = getValoresData(dados_bolsa, dataSeguinte)
        aberturaDiaSeguinte.append(valores['Abertura'])
        fechamentoDiaSeguinte.append(valores['Fechamento'])

        valores = getValoresData(dados_bolsa, dia2)
        abertura2Dias.append(valores['Abertura'])
        fechamento2Dias.append(valores['Fechamento'])

    print('Aqui passou')

    df['aberturaDiaAnterior'] = aberturaDiaAnterior
    df['fechamentoDiaAnterior'] = fechamentoDiaAnterior

    df['aberturaDiaAtual'] = aberturaDiaAtual
    df['fechamentoDiaAtual'] = fechamentoDiaAtual

    df['aberturaDiaSeguinte'] = aberturaDiaSeguinte
    df['fechamentoDiaSeguinte'] = fechamentoDiaSeguinte

    df['abertura2Dias'] = abertura2Dias
    df['fechamento2Dias'] = fechamento2Dias

    return df

In [21]:
for empresa in empresas:
    print(empresa)
    df = extraiValoresBolsa(empresa)
    df.to_csv(empresa + '.csv')

AMER3
Aqui passou
MGLU3
Aqui passou
PETR4
Aqui passou
